## Reconstruct toy features using Method of Optimal Directions (MOD)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dists
import numpy as np
from interp_utils import see, asee
from interp_utils import get_scheduler
from interp_utils import reload_module
from tqdm import tqdm
p=print


In [2]:
reload_module('interp_models')
from interp_models import Autoencoder, SparseAutoencoder, SparseNNMF
from tqdm import tqdm
from interp_utils import get_scheduler

reload_module('toy_models')
from toy_models import SparseIndependent

N_FEATURES = 300
D_MODEL = 100
N_EPOCHS = 10
CODE_STEPS = 1000
ATOM_STEPS=1000
BATCH_SIZE=10000
SPARSE_COEF = 1

toy = SparseIndependent(n_features=N_FEATURES, d_model=D_MODEL, feature_sparsity=0.04)
hidden_state, ground_truth = toy(D_MODEL)

nnmf = SparseNNMF(n_features=N_FEATURES, d_model=D_MODEL)


for epoch in range(N_EPOCHS):
    optimizer = optim.Adam(nnmf.parameters(), lr=1e-2)
    scheduler = get_scheduler(optimizer, CODE_STEPS)

    batch, features = toy(BATCH_SIZE)

    # update codes
    nnmf.train(batch, frozen_atoms=epoch > 1, sparse_coef=SPARSE_COEF, n_steps=CODE_STEPS, reinit_codes=True)

    # update atoms
    nnmf.train(batch, frozen_codes=True, sparse_coef=SPARSE_COEF, n_steps=ATOM_STEPS)



loss: 0.020, mse: 0.007, sparse: 0.013:  32%|███▏      | 322/1000 [00:04<00:08, 75.60it/s]


KeyboardInterrupt: 

In [44]:
# batch, features = toy(1000)

# nnmf.train(batch, n_steps=3000, frozen_atoms=True)



loss: 0.004, mse: 0.001, sparse: 0.032: 100%|██████████| 3000/3000 [00:10<00:00, 293.38it/s]


In [3]:
from interp_utils import hist, heatmap
hist((nnmf.atoms @ toy.normed_features.T).max(dim=-1).values)

In [280]:
from toy_models import Tree
class Direction(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.direction = nn.Parameter(torch.randn(d_model)/np.sqrt(d_model))
    def forward(self, batch):
        return batch @ self.direction
    def norm(self):
        with torch.no_grad():
            self.direction.data.div_(torch.norm(self.direction.data))

# directions = []

# for _ in range(40):
    
toy = SparseIndependent(n_features=N_FEATURES, d_model=D_MODEL, feature_sparsity=0.12)

direction = Direction(D_MODEL)
direction.norm()

optimizer = optim.SGD(direction.parameters(), lr=3e-2)
scheduler = get_scheduler(optimizer, 1000)

pbar = tqdm(range(1000))
for step in pbar:
    batch, features = toy(300)
    dist = direction(batch)
    kurtosis = (((dist - dist.mean())/dist.std())**4).mean()
    # skewness = (((dist - dist.mean())/dist.std())**3).mean()
    # loss = -skewness
    loss = -kurtosis

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    direction.norm()

    pbar.set_description(f'kurtosis: {kurtosis.item():.2f}')
    
    # directions.append({'direction': direction.direction.detach().cpu().numpy(), 'kurtosis': kurtosis.item()})


kurtosis: 3.85: 100%|██████████| 1000/1000 [00:01<00:00, 684.62it/s]


In [234]:
# kurtosis = torch.tensor([item['kurtosis'] for item in directions])
# max_kurtosis = kurtosis.max()
# max_direction = directions[kurtosis.topk(k=1, largest=True).indices[0].item()]['direction']
# max_direction = torch.tensor(direction)

RuntimeError: Could not infer dtype of Direction

In [273]:
hist(direction.direction @ batch.T).show()
hist(direction.direction @ toy.features.T).show()

In [251]:
(direction.direction @ toy.features.T).argmax()

tensor(140)